In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px

MAP

In [ ]:
df_map=pd.read_csv("directory.csv")
df_map.drop(df_map.loc[df_map["Brand"]=="Teavana"].index,inplace=True)
df_map["Phone Number"].fillna("n/a",inplace=True)
df_map["Postcode"].fillna(".",inplace=True)
df_map["Address"]= np.where(df_map["Postcode"]==".",(df_map["Street Address"]+", "+df_map["City"]),(df_map["Street Address"]+", "+df_map["City"]+", "+df_map["Postcode"]))
df_map.drop(["Brand","Store Number","Street Address","City","State/Province","Country","Postcode","Timezone"],axis=1,inplace=True)
df_map=df_map.reindex(columns=["Store Name","Address","Phone Number","Ownership Type","Latitude","Longitude"])

In [ ]:
df_map.head()

map figure:

In [ ]:
figure_map=px.scatter_geo(data_frame=df_map,lat="Latitude",lon="Longitude",color="Ownership Type",color_discrete_map={"Licensed":"#bcbd22","Joint Venture":"#d62728","Company Owned":"#17becf","Franchise":"#1f77b4"},hover_name="Store Name",hover_data={"Address":True,"Phone Number":True,"Ownership Type":False,"Longitude":False,"Latitude":False},template="ggplot2")
figure_map.update_layout(margin={"t":0,"r":0,"b":0,"l":0})
figure_map.update_layout({"plot_bgcolor":"rgba(0,0,0,0)","paper_bgcolor":"rgba(0,0,0,0)"})
figure_map.update_layout(legend={"title":"","orientation":"h","x":0.5,"y":1,"xanchor":"center","bgcolor":"rgba(0,0,0,0)"})

DRINKS

In [ ]:
df=pd.read_csv("starbucks_drinkMenu_expanded.csv")
df["Vitamin A (% DV)"]=df["Vitamin A (% DV)"].apply(lambda x:x.split("%")[0])
df["Vitamin C (% DV)"]=df["Vitamin C (% DV)"].apply(lambda x:x.split("%")[0])
df["Calcium (% DV)"]=df["Calcium (% DV)"].apply(lambda x:x.split("%")[0])
df["Iron (% DV)"]=df["Iron (% DV)"].apply(lambda x:x.split("%")[0])
df[["Total Fat (g)","Vitamin A (% DV)","Vitamin C (% DV)","Calcium (% DV)","Iron (% DV)","Caffeine (mg)"]]=df[["Total Fat (g)","Vitamin A (% DV)","Vitamin C (% DV)","Calcium (% DV)","Iron (% DV)","Caffeine (mg)"]].astype("float")
df["Milk"].fillna("No Milk",inplace=True)

In [ ]:
df.head()

category dropdown:

In [ ]:
df["Category"].unique()

In [ ]:
#category dropdown options
[{"label":i,"value":i} for i in df["Category"].unique()]

beverage dropdown:

In [ ]:
df["Beverage"].unique()

In [ ]:
#@app.callback(
#    Output("beverage_dropdown","options"),
#    [Input("category_dropdown","value")])
def set_beverage_options(selected_category):
    beverage_options=[{"label":i,"value":i} for i in df[df["Category"]==selected_category]["Beverage"].unique()]
    return beverage_options

In [ ]:
set_beverage_options("Classic Espresso Drinks")

In [ ]:
df[df["Category"]=="Classic Espresso Drinks"]["Beverage"].unique()

size dropdown:

In [ ]:
df["Size"].unique()

In [ ]:
#@app.callback(
#    Output("size_dropdown","options"),
#    [Input("beverage_dropdown","value")])
def set_size_options(selected_beverage):
    size_options=[{"label":i,"value":i} for i in df[df["Beverage"]==selected_beverage]["Size"].unique()]
    return size_options

In [ ]:
set_size_options("Espresso")

In [ ]:
df[df["Beverage"]=="Espresso"]["Size"].unique()

milk dropdown:

In [ ]:
df["Milk"].unique()

In [ ]:
#@app.callback(
#    Output("milk_dropdown","options"),
#    [Input("size_dropdown","value"),
#     Input("beverage_dropdown","value")])
def set_milk_options(selected_size,selected_beverage):
    milk_options=[{"label":i,"value":i} for i in df[(df["Size"]==selected_size)&(df["Beverage"]==selected_beverage)]["Milk"].unique()]
    return milk_options

In [ ]:
set_milk_options("Tall","Mocha (Without Whipped Cream)")

In [ ]:
df[(df["Size"]=="Tall")&(df["Beverage"]=="Mocha (Without Whipped Cream)")]["Milk"].unique()

nutrition figure:

In [ ]:
#@app.callback(
#    [Output(component_id="calories_output",component_property="children"),
#     Output(component_id="caffeine_output",component_property="children"),
#     Output(component_id="cholesterol_output",component_property="children"),
#     Output(component_id="nutrition_figure",component_property="figure"),
#     Output(component_id="vita_figure",component_property="figure"),
#     Output(component_id="vitc_figure",component_property="figure"),
#     Output(component_id="calci_figure",component_property="figure"),
#     Output(component_id="iron_figure",component_property="figure")],
#    [Input(component_id="category_dropdown",component_property="value"),
#     Input(component_id="beverage_dropdown",component_property="value"),
#     Input(component_id="size_dropdown",component_property="value"),
#     Input(component_id="milk_dropdown",component_property="value")])
def update_nutritional_figures(selected_category,selected_beverage,selected_size,selected_milk):

    return calories,caffeine,cholesterol,figure_nutrition,figure_vita,figure_vitc,figure_calci,figure_iron

In [ ]:
    if (selected_category is None)|(selected_beverage is None)|(selected_size is None)|(selected_milk is None):
        calories="CALORIES: "
        caffeine="CAFFEINE:  (mg)"
        cholesterol="CHOLESTEROL:  (mg)"

        df_empty=pd.DataFrame([["Total Fat (g)",0],["Trans Fat (g)",0],["Saturated Fat (g)",0],["Sodium (mg)",0],["Total Carbohydrates (g)",0],["Dietary Fibre (g)",0],["Sugars (g)",0],["Protein (g)",0]],columns=["Nutrition","Amount"])
        figure_nutrition=px.bar(data_frame=df_empty,x="Nutrition",y="Amount",color="Nutrition",hover_name="Nutrition",hover_data={"Nutrition":False},range_y=[0,10])
        figure_nutrition.update_layout(xaxis_title="",plot_bgcolor="rgba(0,0,0,0)",paper_bgcolor="rgba(0,0,0,0)")
        figure_nutrition.add_annotation(xref="paper",x=1.35,yref="paper",y=0.09,showarrow=False,text="Click to remove or<br>doubleclick to<br>highlight a nutrition",font={"color":"#c7c7c7"},bordercolor="#c7c7c7",borderwidth=1.1,borderpad=7)

        figure_vita=px.pie(values=[100],color_discrete_sequence=["#ededed"],hole=0.5)
        figure_vita.update_traces(textinfo="none")
        figure_vita.update_layout(hovermode=False,margin={"t":10,"r":10,"b":10,"l":10},plot_bgcolor="rgba(0,0,0,0)",paper_bgcolor="rgba(0,0,0,0)",showlegend=False)
        figure_vita.add_annotation(x=0.5,y=0.5,showarrow=False,text="Vitamin A",font={"size":18})

        figure_vitc=px.pie(values=[100],color_discrete_sequence=["#ededed"],hole=0.5)
        figure_vitc.update_traces(textinfo="none")
        figure_vitc.update_layout(hovermode=False,margin={"t":10,"r":10,"b":10,"l":10},plot_bgcolor="rgba(0,0,0,0)",paper_bgcolor="rgba(0,0,0,0)",showlegend=False)
        figure_vitc.add_annotation(x=0.5,y=0.5,showarrow=False,text="Vitamin C",font={"size":18})

        figure_calci=px.pie(values=[100],color_discrete_sequence=["#ededed"],hole=0.5)
        figure_calci.update_traces(textinfo="none")
        figure_calci.update_layout(hovermode=False,margin={"t":10,"r":10,"b":10,"l":10},plot_bgcolor="rgba(0,0,0,0)",paper_bgcolor="rgba(0,0,0,0)",showlegend=False)
        figure_calci.add_annotation(x=0.5,y=0.5,showarrow=False,text="Calcium",font={"size":18})

        figure_iron=px.pie(values=[100],color_discrete_sequence=["#ededed"],hole=0.5)
        figure_iron.update_traces(textinfo="none")
        figure_iron.update_layout(hovermode=False,margin={"t":10,"r":10,"b":10,"l":10},plot_bgcolor="rgba(0,0,0,0)",paper_bgcolor="rgba(0,0,0,0)",showlegend=False)
        figure_iron.add_annotation(x=0.5,y=0.5,showarrow=False,text="Iron",font={"size":18})

In [ ]:
    else:
        dff=df[(df["Category"]==selected_category)&(df["Beverage"]==selected_beverage)&(df["Size"]==selected_size)&(df["Milk"]==selected_milk)]

        calories="CALORIES: {}".format(int(dff["Calories"]))
        caffeine="CAFFEINE: {} (mg)".format(int(dff["Caffeine (mg)"]))
        cholesterol="CHOLESTEROL: {} (mg)".format(int(dff["Cholesterol (mg)"]))

        df_nutrition=dff[["Total Fat (g)","Trans Fat (g)","Saturated Fat (g)","Sodium (mg)","Total Carbohydrates (g)","Dietary Fibre (g)","Sugars (g)","Protein (g)"]]
        df_nutrition=df_nutrition.melt(var_name="Nutrition",value_name="Amount")
        figure_nutrition=px.bar(data_frame=df_nutrition,x="Nutrition",y="Amount",color="Nutrition",hover_name="Nutrition",hover_data={"Nutrition":False})
        figure_nutrition.update_layout(xaxis_title="",plot_bgcolor="rgba(0,0,0,0)",paper_bgcolor="rgba(0,0,0,0)")
        figure_nutrition.add_annotation(xref="paper",x=1.35,yref="paper",y=0.09,showarrow=False,text="Click to remove or<br>doubleclick to<br>highlight a nutrition",font={"color":"#c7c7c7"},bordercolor="#c7c7c7",borderwidth=1.1,borderpad=7)

        df_vita=pd.DataFrame(data=[int(dff["Vitamin A (% DV)"]),100-int(dff["Vitamin A (% DV)"])],index=["Vit","Nil"],columns=["Percent"])
        figure_vita=px.pie(data_frame=df_vita,names=df_vita.index,values="Percent",color=df_vita.index,color_discrete_map={df_vita.index[0]:"yellow",df_vita.index[1]:"#ededed"},hole=0.5)
        figure_vita.update_traces(sort=False,textposition="outside")
        figure_vita.update_layout(hovermode=False,margin={"t":10,"r":10,"b":10,"l":10},plot_bgcolor="rgba(0,0,0,0)",paper_bgcolor="rgba(0,0,0,0)",showlegend=False)
        figure_vita.add_annotation(x=0.5,y=0.5,showarrow=False,text="Vitamin A",font={"size":18})

        df_vitc=pd.DataFrame(data=[int(dff["Vitamin C (% DV)"]),100-int(dff["Vitamin C (% DV)"])],index=["Vit","Nil"],columns=["Percent"])
        figure_vitc=px.pie(data_frame=df_vitc,names=df_vitc.index,values="Percent",color=df_vitc.index,color_discrete_map={df_vitc.index[0]:"yellow",df_vitc.index[1]:"#ededed"},hole=0.5)
        figure_vitc.update_traces(sort=False,textposition="outside")
        figure_vitc.update_layout(hovermode=False,margin={"t":10,"r":10,"b":10,"l":10},plot_bgcolor="rgba(0,0,0,0)",paper_bgcolor="rgba(0,0,0,0)",showlegend=False)
        figure_vitc.add_annotation(x=0.5,y=0.5,showarrow=False,text="Vitamin C",font={"size":18})

        df_calci=pd.DataFrame(data=[int(dff["Calcium (% DV)"]),100-int(dff["Calcium (% DV)"])],index=["Vit","Nil"],columns=["Percent"])
        figure_calci=px.pie(data_frame=df_calci,names=df_calci.index,values="Percent",color=df_calci.index,color_discrete_map={df_calci.index[0]:"yellow",df_calci.index[1]:"#ededed"},hole=0.5)
        figure_calci.update_traces(sort=False,textposition="outside")
        figure_calci.update_layout(hovermode=False,margin={"t":10,"r":10,"b":10,"l":10},plot_bgcolor="rgba(0,0,0,0)",paper_bgcolor="rgba(0,0,0,0)",showlegend=False)
        figure_calci.add_annotation(x=0.5,y=0.5,showarrow=False,text="Calcium",font={"size":18})

        df_iron=pd.DataFrame(data=[int(dff["Iron (% DV)"]),100-int(dff["Iron (% DV)"])],index=["Vit","Nil"],columns=["Percent"])
        figure_iron=px.pie(data_frame=df_iron,names=df_iron.index,values="Percent",color=df_iron.index,color_discrete_map={df_iron.index[0]:"yellow",df_iron.index[1]:"#ededed"},hole=0.5)
        figure_iron.update_traces(sort=False,textposition="outside")
        figure_iron.update_layout(hovermode=False,margin={"t":10,"r":10,"b":10,"l":10},plot_bgcolor="rgba(0,0,0,0)",paper_bgcolor="rgba(0,0,0,0)",showlegend=False)
        figure_iron.add_annotation(x=0.5,y=0.5,showarrow=False,text="Iron",font={"size":18})